<a href="https://colab.research.google.com/github/a-mhamdi/nlp-unraveled/blob/main/05_text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Analysis and Advanced Techniques

**Outlines**
---
***
---

## Text classification


## Named Entity Recognition (NER)


## Sentiment analysis


## Word embeddings